In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm, jarque_bera, kstest, gamma, expon

# 1. Parametric: Bootstrapping

In [2]:
df = pd.read_csv("ScanRecords.csv")
df['Date'] = pd.to_datetime(df['Date'])
print("Shape of df: ", df.shape)
df.head()

Shape of df:  (618, 4)


,Date,Time,Duration,PatientType
0,2023-08-01,8.23,0.949176,Type 2
1,2023-08-01,8.49,0.479593,Type 1
2,2023-08-01,9.12,0.496112,Type 2
3,2023-08-01,10.26,0.691947,Type 2
4,2023-08-01,10.64,0.345412,Type 1


In [3]:
df1 = df[df["PatientType"] == "Type 1"]
df2 = df[df["PatientType"] == "Type 2"]

In [4]:
print(df.groupby('PatientType')['Duration'].describe())  

             count      mean       std       min       25%       50%  \
PatientType                                                            
Type 1       379.0  0.432661  0.097774  0.093731  0.368123  0.435888   
Type 2       239.0  0.669339  0.187286  0.220708  0.523962  0.646603   

                  75%       max  
PatientType                      
Type 1       0.496907  0.708922  
Type 2       0.796523  1.146789  


## Type 1

## 1.1. Duration (Normal)

In [23]:
df1.head()

,Date,Time,Duration,PatientType
1,2023-08-01,8.49,0.479593,Type 1
4,2023-08-01,10.64,0.345412,Type 1
5,2023-08-01,11.07,0.422270,Type 1
6,2023-08-01,11.13,0.356129,Type 1
8,2023-08-01,11.56,0.423303,Type 1


In [25]:
def bootstrap_Type1(df1, B1, alpha):

    n1 = len(df1)
    X_bar = np.mean(df1['Duration'])
    St_Dev = np.std(df1['Duration'])

    # Initialize empty arrays for bootstrap quantities
    X_star_bar = np.empty(B1)
    X_star_sd = np.empty(B1)
    Q_star = np.empty(B1)

    # bootstrapping
    for b in range(B1):
        J1 = np.random.choice(np.arange(n1), size=n1, replace=True) # Resampling with replacement
        X_star = df1['Duration'].iloc[J1] # Construct the bootstrap sample
        X_star_bar[b] = np.mean(X_star) # Calculate the bootstrap sample mean
        X_star_sd[b] = np.std(X_star) # Calculate the bootstrap sample variance
        Q_star[b] = np.sqrt(n1) * (X_star_bar[b] - X_bar) / X_star_sd[b] # Calculate the bootstrap quantity

    # Critical values
    cv_LB = np.quantile(Q_star, alpha / 2)
    cv_UB = np.quantile(Q_star, 1 - alpha / 2)

    # Confidence interval
    CI_LB = X_bar - cv_LB * St_Dev / np.sqrt(n1)
    CI_UP = X_bar - cv_UB * St_Dev / np.sqrt(n1)

    results = {
        "Bootstrap mean of Duration": np.mean(X_star_bar),
        "95% CI for Mean Duration": (CI_LB, CI_UP),
        "Bootstrap std of Duration": np.mean(X_star_sd) ,
        "95% CI for std Duration": (cv_LB, cv_UB)
    }
    
    return results

results = bootstrap_Type1(df1, B1=499, alpha=0.05)
for key, value in results.items():
    print(f"{key}: {value}")

Bootstrap mean of Duration: 0.43252118884419655
95% CI for Mean Duration: (np.float64(0.4420646488477063), np.float64(0.42306011545002176))
Bootstrap std of Duration: 0.09741698230455403
95% CI for std Duration: (np.float64(-1.8748848753183738), np.float64(1.9141278238596104))


In [ ]:
## TODO: ADD QUANTILES W CI

## 1.2. Mean Interarrival time (exponential)

In [30]:
numRows = len(df1)
interArrivals1 = []

for i in range(numRows - 1):
    if df1.iloc[i]["Date"] == df1.iloc[i + 1]["Date"]:
        # interaarival time of arrivals on the same day
        interval = df1.iloc[i + 1]["Time"] - df1.iloc[i]["Time"]
    else:
        interval = (17 - df1.iloc[i]["Time"]) + (df1.iloc[i + 1]["Time"] - 8)
    interArrivals1.append(interval)

interArrivals1 = np.array(interArrivals1)

print("Mean Inter-arrival time: ", np.mean(interArrivals1))

Mean Inter-arrival time:  0.5453439153439154


In [ ]:
def bootstrap_Type1(interArrivals, B1, alpha):

    n1 = len(interArrivals)
    X_bar = np.mean(interArrivals)
    St_Dev = np.std(interArrivals)

    # empty arrays for bootstrap quantities
    X_star_bar = np.empty(B1)
    X_star_sd = np.empty(B1)
    Q_star = np.empty(B1)

    # bootstrapping
    for b in range(B1):
        J1 = np.random.choice(np.arange(n1), size=n1, replace=True) # Resampling with replacement
        X_star = interArrivals[J1] # Construct the bootstrap sample
        X_star_bar[b] = np.mean(X_star) # Calculate the bootstrap sample mean
        X_star_sd[b] = np.std(X_star) # Calculate the bootstrap sample variance
        Q_star[b] = np.sqrt(n1) * (X_star_bar[b] - X_bar) / X_star_sd[b] # Calculate the bootstrap quantity

    # Critical values
    cv_LB = np.quantile(Q_star, alpha / 2)
    cv_UB = np.quantile(Q_star, 1 - alpha / 2)

    # Confidence interval
    CI_LB = X_bar - cv_LB * St_Dev / np.sqrt(n1)
    CI_UP = X_bar - cv_UB * St_Dev / np.sqrt(n1)

    results = {
        "Bootstrap mean of interArrivals": np.mean(X_star_bar),
        "95% CI for Mean interArrivals": (CI_LB, CI_UP),
        "Bootstrap std of interArrivals": np.mean(X_star_sd),
        "95% CI for std interArrivals": (cv_LB, cv_UB)
    }
    
    return results

results = bootstrap_Type1(interArrivals1, B1=499, alpha=0.05)
for key, value in results.items():
    print(f"{key}: {value}")

Bootstrap mean of interArrivals: 0.5446245400854618
95% CI for Mean interArrivals: (np.float64(0.6203758653854785), np.float64(0.4895444229586992))
Bootstrap std of interArrivals: 0.5795309007316473
95% CI for std interArrivals: (np.float64(-2.50422752248658), np.float64(1.8623349718944266))


# 2. Type 2

## 2.1. Duration (Gamma)

In [11]:
shape_b, loc_b, scale_b = gamma.fit(df2['Duration'], floc=0) 
print(shape_b)
print(scale_b)

12.584814582926688
0.05318623207452434


In [31]:
# Set the seed
np.random.seed(515)

data = df2['Duration'].values  

# Parameters
nr_sim = 1000  # Number of simulations
B = 499  # Number of bootstrap replications
n2 = len(data)  # Sample size
alpha = 0.05  # Nominal level of the test
reject = np.zeros(nr_sim)  # Vector to store rejections

# initial statistic
xbar = np.mean(data)
std = np.std(data, ddof=1)

# mc simulation
for i in range(nr_sim):

    X = np.random.choice(data, size=n2, replace=True)  # Draw X from the actual data with replacement

    X_bar = np.mean(X)  # Sample mean of X
    St_Dev = np.std(X, ddof=1)  # Standard deviation of X (ddof=1 for sample std)
    Q = np.sqrt(n2) * (X_bar - xbar) / St_Dev  # Test statistic

    # Empty arrays for bootstrap quantities
    Q_star = np.empty(B)
    gamma_shape = np.empty(B)
    gamma_scale = np.empty(B)

    for b in range(B):
        J = np.random.choice(n2, size=n2, replace=True)  # Draw the indices J
        X_star = X[J]  # Draw the bootstrap sample
        
        # Bootstrap sample statistics
        X_bar_star = np.mean(X_star)  # Bootstrap sample mean
        St_Dev_star = np.std(X_star, ddof=1)  # Bootstrap standard deviation
        Q_star[b] = np.sqrt(n2) * (X_bar_star - X_bar) / St_Dev_star  # Bootstrap test statistic
        
        # Fit gamma distribution to bootstrap sample
        shape_b, loc_b, scale_b = gamma.fit(X_star, floc=0)  # Fix location to 0
        gamma_shape[b] = shape_b
        gamma_scale[b] = scale_b

    # Critical values
    cv_star = np.quantile(Q_star, 1 - alpha)  # Bootstrap critical value
    
    cv_LB = np.quantile(Q_star, alpha / 2)  # Lower bound critical value
    cv_UB = np.quantile(Q_star, 1 - alpha / 2)  # Upper bound critical value

    # Confidence interval
    CI_LB = X_bar - cv_UB * std / np.sqrt(n2)
    CI_UP = X_bar - cv_LB * std / np.sqrt(n2)


    # Step 3: Evaluate
    if Q < cv_LB or Q > cv_UB:  # Two-sided test:
        reject[i] = 1  # Check if the null hypothesis is rejected

ERF = np.mean(reject)  # Empirical rejection frequency

# Bootstrap Confidence Intervals for Gamma Shape and Scale
shape_CI = np.percentile(gamma_shape, [alpha / 2 * 100, (1 - alpha / 2) * 100])
scale_CI = np.percentile(gamma_scale, [alpha / 2 * 100, (1 - alpha / 2) * 100])

# Output
print(f"Rejection: {100 * ERF:}%")
print(f"Bootstrap Confidence Interval for Shape: ({shape_CI[0]:.4f}, {shape_CI[1]:.4f})")
print(f"Bootstrap Confidence Interval for Scale: ({scale_CI[0]:.4f}, {scale_CI[1]:.4f})")
print(f"Confidence Interval for Mean: ({CI_LB:.4f}, {CI_UP:.4f})")


Rejection: 3.5999999999999996%
Bootstrap Confidence Interval for Shape: (10.6488, 14.5411)
Bootstrap Confidence Interval for Scale: (0.0469, 0.0645)
Confidence Interval for Mean: (0.6585, 0.7080)


In [21]:
print(f"Simulation {i + 1}: Bootstrap Gamma Shape: {np.mean(gamma_shape):.4f}, Scale: {np.mean(gamma_scale):.4f}")

Simulation 1000: Bootstrap Gamma Shape: 12.3905, Scale: 0.0556


## 2.2. Interarrival (Normal)

In [5]:
numRows = len(df2)
interArrivals2 = []

for i in range(numRows - 1):
    if df2.iloc[i]["Date"] == df2.iloc[i + 1]["Date"]:
        # interaarival time of arrivals on the same day
        interval = df2.iloc[i + 1]["Time"] - df2.iloc[i]["Time"]
    else:
        interval = (17 - df2.iloc[i]["Time"]) + (df2.iloc[i + 1]["Time"] - 8)
    interArrivals2.append(interval)

interArrivals2 = np.array(interArrivals2)

In [ ]:
# Define the Monte Carlo simulation for Type 2 patients
def monte_carlo_Type2(n, alpha, mu, gamma_shape, gamma_scale, nr_sim):
    np.random.seed(515)  # Set the seed for reproducibility

    reject_n = np.zeros(nr_sim)  # Store rejections for normal critical value
    reject_t = np.zeros(nr_sim)  # Store rejections for t critical value

    for i in range(nr_sim):
        # Step 1: Simulate
        X = np.random.gamma(shape=gamma_shape, scale=gamma_scale, size=n)  # Gamma-distributed random values

        # Step 2: Apply
        X_bar = np.mean(X)  # Sample mean of X
        St_Dev = np.std(X)  # Standard deviation of X
        Q_n = np.sqrt(n) * X_bar / St_Dev  # Test statistic

        cv_n = np.quantile(np.random.normal(0, 1, 10000), 1 - alpha)  # Normal critical value
        cv_t = np.quantile(np.random.standard_t(df=n - 1, size=10000), 1 - alpha)  # t critical value

        # Step 3: Evaluate
        if Q_n > cv_n:
            reject_n[i] = 1  # Reject using normal critical value
        if Q_n > cv_t:
            reject_t[i] = 1  # Reject using t critical value

    results = {
        "Rejection rate (Normal critical value)": np.mean(reject_n),
        "Rejection rate (t critical value)": np.mean(reject_t)
    }

    return results


# Example usage for Type 2 patients
gamma_shape = 2  # Example shape parameter for gamma distribution
gamma_scale = 1  # Example scale parameter for gamma distribution
results_type2 = monte_carlo_Type2(nr_sim=5000, n=100, alpha=0.05, mu=0, gamma_shape=gamma_shape, gamma_scale=gamma_scale)
for key, value in results_type2.items():
    print(f"{key}: {value}")


In [ ]:
# Fit a gamma distribution to the data
gamma_params = gamma.fit(interArrivals2, floc=0)  # floc=0 fixes the location parameter to 0
gamma_shape, loc, gamma_scale = gamma_params # shape = alpha, scale = beta

print(f"MLE gamma_shape: {gamma_shape}")
print(f"MLE gamma_scale: {gamma_scale}")


In [27]:
import numpy as np
from scipy.stats import gamma
from scipy.stats import bootstrap
from scipy.stats import describe

# Set seed
np.random.seed(515)

# Data and parameters
data = df2['Duration'].values  # Assuming this is a pandas DataFrame
nr_sim = 1000  # Number of Monte Carlo simulations
B1 = 499  # Number of bootstrap replications
n2 = len(data)  # Sample size
alpha = 0.05  # Nominal level of the test

# Initial statistics
X_bar3 = np.mean(data)
St_Dev3 = np.std(data, ddof=1)

# Pre-allocate arrays
X_star_bar3 = np.empty(B1)
X_star_sd3 = np.empty(B1)
X_star_gamma_shape = np.empty(B1)
X_star_gamma_rate = np.empty(B1)

CI_lower1 = np.empty(nr_sim)
CI_upper1 = np.empty(nr_sim)
reject1 = np.zeros(nr_sim)

# Monte Carlo simulation
for m in range(nr_sim):
    indices = np.random.choice(n2, size=n2, replace=True)  # Sample indices with replacement
    sim_sample = data[indices]  # Simulate sample
    sim_sample_mean = np.mean(sim_sample)
    Q = np.sqrt(n2) * (sim_sample_mean - X_bar3) / np.std(sim_sample, ddof=1)

    Q_star3 = np.empty(B1)  # Bootstrap test statistic array

    # Bootstrap procedure
    for b in range(B1):
        J3 = np.random.choice(n2, size=n2, replace=True)  # Bootstrap sample indices
        X_star3 = sim_sample[J3]  # Bootstrap sample
        X_star_bar3[b] = np.mean(X_star3)
        X_star_sd3[b] = np.std(X_star3, ddof=1)
        fit_gamma_star_shape, fit_gamma_star_loc, fit_gamma_star_scale = gamma.fit(X_star3, floc=0)
        X_star_gamma_shape[b] = fit_gamma_star_shape
        X_star_gamma_rate[b] = 1 / fit_gamma_star_scale
        Q_star3[b] = np.sqrt(n2) * (X_star_bar3[b] - sim_sample_mean) / X_star_sd3[b]

    # Critical values
    cv_star_lower1 = np.quantile(Q_star3, alpha / 2)
    cv_star_upper1 = np.quantile(Q_star3, 1 - alpha / 2)

    # Confidence interval
    CI_lower1[m] = sim_sample_mean - cv_star_upper1 * St_Dev3 / np.sqrt(n2)
    CI_upper1[m] = sim_sample_mean - cv_star_lower1 * St_Dev3 / np.sqrt(n2)

    # Rejection condition
    if X_bar3 < CI_lower1[m] or X_bar3 > CI_upper1[m]:
        reject1[m] = 1

# Empirical rejection frequency
ERF1 = np.mean(reject1)
print(f"Rejection occurred in {100 * ERF1:.2f}% of the cases.")

# Main bootstrap results
t2_avg_duration = np.mean(X_star_bar3)
t2_sd_duration = np.mean(X_star_sd3)

t2_avg_gamma_shape = np.mean(X_star_gamma_shape)
t2_avg_gamma_rate = np.mean(X_star_gamma_rate)

print(f"Avg duration: {t2_avg_duration:.6f}")
print(f"SD duration: {t2_sd_duration:.6f}")
print(f"Avg gamma shape: {t2_avg_gamma_shape:.6f}")
print(f"Avg gamma rate: {t2_avg_gamma_rate:.6f}")

# Critical values
cv_lower3 = np.quantile(Q_star3, alpha / 2)
cv_upper3 = np.quantile(Q_star3, 1 - alpha / 2)

# Confidence interval
CI_dur_t2_lower = X_bar3 - cv_upper3 * St_Dev3 / np.sqrt(n2)
CI_dur_t2_upper = X_bar3 - cv_lower3 * St_Dev3 / np.sqrt(n2)

print(f"CI for duration (lower): {CI_dur_t2_lower:.6f}")
print(f"CI for duration (upper): {CI_dur_t2_upper:.6f}")


Rejection occurred in 3.30% of the cases.
Avg duration: 0.684425
SD duration: 0.192227
Avg gamma shape: 12.390541
Avg gamma rate: 18.105685
CI for duration (lower): 0.643840
CI for duration (upper): 0.693361


In [28]:
import numpy as np
from scipy.stats import gamma

# Set the seed
np.random.seed(515)

data = df2['Duration'].values  # Assuming df2 is a pandas DataFrame

# Parameters
nr_sim = 1000  # Number of simulations
B = 499  # Number of bootstrap replications
n2 = len(data)  # Sample size
alpha = 0.05  # Nominal level of the test
reject = np.zeros(nr_sim)  # Vector to store rejections

# Initial statistics
xbar = np.mean(data)
std = np.std(data, ddof=1)

# Start the simulations
for i in range(nr_sim):
    # Generate a bootstrap sample
    X = np.random.choice(data, size=n2, replace=True)

    # Calculate sample mean and standard deviation
    X_bar = np.mean(X)
    St_Dev = np.std(X, ddof=1)

    # Calculate the test statistic
    Q = np.sqrt(n2) * (X_bar - xbar) / St_Dev

    # Empty arrays for bootstrap quantities
    Q_star = np.empty(B)
    gamma_shape = np.empty(B)
    gamma_scale = np.empty(B)

    # Bootstrap procedure
    for b in range(B):
        # Generate a bootstrap sample from X
        J = np.random.choice(n2, size=n2, replace=True)
        X_star = X[J]

        # Calculate bootstrap sample statistics
        X_bar_star = np.mean(X_star)
        St_Dev_star = np.std(X_star, ddof=1)
        Q_star[b] = np.sqrt(n2) * (X_bar_star - X_bar) / St_Dev_star

        # Fit gamma distribution to the bootstrap sample
        shape_b, loc_b, scale_b = gamma.fit(X_star, floc=0)  # Fix location to 0
        gamma_shape[b] = shape_b
        gamma_scale[b] = scale_b

    # Critical values
    cv_LB = np.quantile(Q_star, alpha / 2)  # Lower bound critical value
    cv_UB = np.quantile(Q_star, 1 - alpha / 2)  # Upper bound critical value

    # Confidence interval for the mean
    CI_LB = X_bar - cv_UB * std / np.sqrt(n2)
    CI_UP = X_bar - cv_LB * std / np.sqrt(n2)

    # Two-sided hypothesis test
    if Q < cv_LB or Q > cv_UB:
        reject[i] = 1  # Reject the null hypothesis

# Empirical rejection frequency
ERF = np.mean(reject)

# Bootstrap Confidence Intervals for Gamma Shape and Scale
shape_CI = np.percentile(gamma_shape, [alpha / 2 * 100, (1 - alpha / 2) * 100])
scale_CI = np.percentile(gamma_scale, [alpha / 2 * 100, (1 - alpha / 2) * 100])

# Output
print(f"Rejection occurred in {100 * ERF:.2f}% of the cases.")
print(f"Bootstrap Confidence Interval for Shape: ({shape_CI[0]:.4f}, {shape_CI[1]:.4f})")
print(f"Bootstrap Confidence Interval for Scale: ({scale_CI[0]:.4f}, {scale_CI[1]:.4f})")
print(f"Confidence Interval for Mean: ({CI_LB:.4f}, {CI_UP:.4f})")


Rejection occurred in 3.60% of the cases.
Bootstrap Confidence Interval for Shape: (10.6488, 14.5411)
Bootstrap Confidence Interval for Scale: (0.0469, 0.0645)
Confidence Interval for Mean: (0.6585, 0.7080)
